# Steepest descent

In [ ]:
using Plots, ForwardDiff, Printf, LinearAlgebra

# Steepest descent on quadratic with exact linesearch

In [ ]:
#f(x) = 1/2 x'Ax + b'x (ignoring constant terms)
function grad_method_exact_ls(A, b, x0, ϵ=1e-6)
    x = copy(x0)
    ∇f = A*x + b
    k = 0
    xtrace = x'
    while norm(∇f) > ϵ
        α = dot(∇f,∇f) / dot(∇f,A*∇f)
        x = x - α*∇f
        ∇f = A*x + b
        f = (1/2)x'*A*x + b'*x
        @printf "it = %3d | |∇f| = %8.2e | f = %8.2e\n" k norm(∇f) f
        k += 1
        xtrace = vcat(xtrace,x')
    end
    return xtrace
end


In [ ]:
#Apply to f(x) = x^2+2y^2
A = [1 0;0 2]
b = [0, 0]
x0 = [2, 1]
xtrace = grad_method_exact_ls(A, b, x0);

In [ ]:
# contour plot
f(x1,x2) = (1/2)*[x1, x2]'*A*[x1, x2]+b'*[x1,x2]
x1 = -2:0.05:2
x2 = -2:0.05:2;

contour(x1, x2, f, levels = 50)
plot!(xtrace[:,1], xtrace[:,2], marker = 3)


In [ ]:
#Apply to f(x) = x^2 + y^2/100
A = [1 0; 0 1/100]
b = [0, 0]
x0 = [2 ,1]
f(x) = x'*A*x + b'*x
∇f(x) = A*x + b
xtrace = grad_method_exact_ls(A, b, x0);

In [ ]:
f(x1,x2) = x1^2 + x2^2/100
x1 = -2:0.05:2
x2 = -2:0.05:2
contour(x1, x2, f, levels = 50)
plot!(xtrace[:,1],xtrace[:,2],marker = 3)